# Data Cleaning Listado de Listado de Beneficiarios 2021

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. beneficiarios_21 -> Dataset con el listado de TODOS los beneficiarios de 2021.
2. Estados_Beneficiarios_2019_2022 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text) #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura y limpieza de datos

### 1.1 Beneficiarios 2021

Obtenemos el listado único de localidades y listado único de municipios con sus claves.

In [3]:
beneficiarios_21 = pd.read_csv('../../data/productores_beneficiarios 2019-2022/listado_beneficiarios_fertilizantes_2021.csv', encoding='utf-8', skiprows=0)

In [4]:
beneficiarios_21['ENTIDAD'].unique()

array(['GUERRERO', 'PUEBLA', 'MORELOS', 'TLAXCALA', nan], dtype=object)

In [5]:
beneficiarios_21.dropna(inplace=True)

In [6]:
# Obtenemos las localidades únicas en el dataset.
Localidades_21 = beneficiarios_21[['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD']]
Localidades_21 = Localidades_21.drop_duplicates()


In [7]:
Localidades_21['ENTIDAD_c_benef'] = Localidades_21['ENTIDAD'].apply(clean_text)
Localidades_21['MUNICIPIO_c_benef'] = Localidades_21['MUNICIPIO'].apply(clean_text)
Localidades_21['LOCALIDAD_c_benef'] = Localidades_21['LOCALIDAD'].apply(clean_text)

In [8]:
# Creamos las dos keys de beneficiarios 2019
Localidades_21['KEY_benef_mun'] = Localidades_21['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_21[
    'MUNICIPIO_c_benef'].astype(str)
Localidades_21['KEY_benef_loc'] = Localidades_21['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_21[
    'MUNICIPIO_c_benef'].astype(str) + '-' + Localidades_21['LOCALIDAD_c_benef'].astype(str)

In [9]:
Localidades_21.shape

(5914, 8)

In [10]:
# Obtenemos las localidades únicas en el dataset.
Municipios_21 = beneficiarios_21[['ENTIDAD', 'MUNICIPIO']]
Municipios_21 = Municipios_21.drop_duplicates()

In [11]:
# Estandarizamos la limpieza de los datos
Municipios_21['ENTIDAD_c_benef'] = Municipios_21['ENTIDAD'].apply(clean_text)
Municipios_21['MUNICIPIO_c_benef'] = Municipios_21['MUNICIPIO'].apply(clean_text)

In [12]:
# Creamos las dos keys de beneficiarios 
Municipios_21['KEY_benef_mun'] = Municipios_21['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_21[
    'MUNICIPIO_c_benef'].astype(str)

In [13]:
Municipios_21.shape

(248, 5)

In [14]:
Municipios_21.columns

Index(['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef',
       'KEY_benef_mun'],
      dtype='object')

### 1.2 INEGI 2021

Obtener listado único de municipios y listado único de localidades de inegi de 2021 con sus claves.

In [15]:
path_dataset_inegi_2021 = '../../data/dataset_inegi_clean_2021.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2021 = pd.read_csv(path_dataset_inegi_2021)

### 1.2.1 INEGI 2021 Municipios únicos para cada año.

In [16]:
dataset_inegi_2021['KEY_inegi_municipio'] = dataset_inegi_2021['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2021[
    'Municipio_c_inegi'].astype(str) 
dataset_inegi_2021['KEY_inegi_localidad'] = dataset_inegi_2021['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2021[
    'Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2021['Localidad_c_inegi'].astype(str)

In [17]:
INEGI_UNIQUEMUN_2021 = dataset_inegi_2021.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad"])

INEGI_UNIQUEMUN_2021 = INEGI_UNIQUEMUN_2021.drop_duplicates()

In [18]:
INEGI_UNIQUEMUN_2021.shape

(2471, 7)

In [19]:
INEGI_UNIQUEMUN_2021.columns

Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi',
       'Entidad_c_inegi', 'Municipio_c_inegi', 'KEY_inegi_municipio'],
      dtype='object')

### 1.2.1 INEGI 2021 Localidades únicas para cada año.

In [20]:
INEGI_UNIQUELOC_2021 = dataset_inegi_2021

INEGI_UNIQUELOC_2021 = INEGI_UNIQUELOC_2021.drop_duplicates()

In [21]:
INEGI_UNIQUELOC_2021.shape

(300350, 11)

# 2. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2019

El objetivo de esta sección es crear dos diccionarios de códigos según BENEFICIARIOS-MUNICIPIOS_INEGI y otro BENEFICIARIOS-LOCALIDADES_INEGI, para cada uno de los estados encontrados en inegi_2021, en este caso el número de keys es menor con lo cual no nos hará falta dividir por estado.

Para ello haremos un Left join entre Localidades_21 y el dataset de INEGI correspondiente.

In [22]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 85 y 100 de coincidencia
def fuzzy_merge_benef2019_2022(df_benef, df_inegi, key1, key2, threshold=85, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_inegi[key2].tolist()
    
    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_benef[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))


    # Crear una columna con las mejores coincidencias
    df_benef['best_match'] = [match[0] if match else None for match in matches]
    
    df_benef['match_score'] = [match[1] if match else None for match in matches]
    

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_benef, df_inegi, left_on='best_match', right_on=key2, how='left',
                         suffixes=('_benef', '_inegi'))
    
    return df_merged

In [23]:
diccionario_MUN_21 = fuzzy_merge_benef2019_2022(Municipios_21, INEGI_UNIQUEMUN_2021, 'KEY_benef_mun', 'KEY_inegi_municipio')
diccionario_MUN_21.shape

(248, 14)

In [24]:
diccionario_MUN_21.drop(columns=['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef', 'Entidad_c_inegi',
       'Municipio_c_inegi'], inplace=True)

In [25]:
diccionario_MUN_21.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_21.csv', index=False)

In [26]:
nan_rows = beneficiarios_21[beneficiarios_21.isna().any(axis=1)]

In [27]:
nan_rows

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA


In [28]:
# Assuming your DataFrame is named df
filtered_df = beneficiarios_21[beneficiarios_21['ENTIDAD'] == 'NACIONAL']

In [29]:
filtered_df

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA
